In [1]:
import keyring
import os

os.environ['OPENAI_API_KEY'] = keyring.get_password('openai', 'key_for_windows')
os.environ['ACTIVELOOP_TOKEN'] = keyring.get_password('activeloop', 'key_for_windows')
os.environ['COHERE_API_KEY'] = keyring.get_password('cohere', 'key_for_windows')

In [2]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader('./data/paul_graham/').load_data()

In [3]:
# chunk the documents
# ServiceContext deprecated
# from llama_index.core import ServiceContext

# # initialize service context (set chunk size)
# service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=64)
# node_parser = service_context.node_parser

# nodes = node_parser.get_nodes_from_documents(documents)
# from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

# configure the text splitter with desired chunk size and overlap
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)

# set the text splitter in the global settings
# Settings.text_splitter = text_splitter

# use the text splitter to parse documents into nodes
nodes = []
for doc in documents:
    # split the text of each document
    text_chunks = text_splitter.split_text(doc.text)
    # create nodes from the text chunks
    for chunk in text_chunks:
        node = Document(text=chunk, metadata=doc.metadata)
        nodes.append(node)

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding

my_activeloop_org_id = 'ahn283'
my_activeloop_dataset_name = 'LlamaIndex_paulgraham_essay'
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# initialize embedding function
embedding_function = OpenAIEmbedding()

# create an index over the documents
vector_store = DeepLakeVectorStore(
    dataset_path=dataset_path, 
    embedding_function=embedding_function,
    overwrite=True
)

d:\projects\github\learning\llm\venv_llm\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!


In [5]:
from llama_index.core.storage.storage_context import StorageContext

# intialize StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)

In [6]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(
    documents=nodes, 
    storage_context=storage_context,
    # embedding_function=embedding_function
)

Retrying llama_index.embeddings.openai.base.get_embeddings in 0.6151885792275462 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# SubQuestion Query Engine